In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [2]:
wr = pd.read_csv('../input/wr/dk_wr_stats.csv')
week1 = pd.read_csv('../input/wr/fantasy-football-weekly-projections (2).csv')
week1_def = pd.read_csv('../input/defense_2018_projections.csv')

In [3]:
week1.columns

Index(['Rank', 'Name', 'Team', 'Position', 'Week', 'Opponent',
       'ReceivingTargets', 'Receptions', 'ReceptionPercentage',
       'ReceivingYards', 'ReceivingTouchdowns', 'ReceivingLong',
       'ReceivingYardsPerTarget', 'ReceivingYardsPerReception',
       'RushingAttempts', 'RushingYards', 'RushingYardsPerAttempt',
       'RushingTouchdowns', 'Fumbles', 'FumblesLost', 'FantasyPointsPerGame',
       'FantasyPoints'],
      dtype='object')

In [4]:
wr.drop('ReceivingLong', axis=1,inplace=True)

In [5]:
wr.tail()

,Name,Team,Week,Opponent,ReceivingTargets,Receptions,ReceptionPercentage,ReceivingYards,ReceivingTouchdowns,ReceivingYardsPerTarget,ReceivingYardsPerReception,RushingAttempts,RushingYards,RushingYardsPerAttempt,RushingTouchdowns,Fumbles,FumblesLost,points,year
14925,Tavon Austin,LAR,17,SF,5,3,60.0,14,0,2.8,4.7,4,24,6.0,0,0,0,6.8,2017
14926,Allen Hurns,JAX,17,TEN,5,3,60.0,38,0,7.6,12.7,0,0,0.0,0,0,0,6.8,2017
14927,Michael Campanaro,BAL,17,CIN,4,3,75.0,36,0,9.0,12.0,0,0,0.0,0,0,0,6.6,2017
14928,Kendall Wright,CHI,17,MIN,8,5,62.5,29,0,3.6,5.8,0,0,0.0,0,0,0,7.9,2017
14929,Jaydon Mickens,JAX,17,TEN,0,0,0.0,0,0,0.0,0.0,0,0,0.0,0,2,1,-1.0,2017


## WR Weekly projections dataframe feature engineering 

In [7]:
week1.drop(['Rank','Position','ReceivingLong','FantasyPointsPerGame'], axis=1, inplace=True)

week1['year'] =2018
week1.columns = wr.columns

indices = []
for idx,player in enumerate(week1['Name']):
    if player not in wr['Name'].unique():
        indices.append(idx)

week1.drop(indices, axis=0, inplace=True)
week1.reset_index(inplace=True, drop=True)

In [8]:
wr.columns

Index(['Name', 'Team', 'Week', 'Opponent', 'ReceivingTargets', 'Receptions',
       'ReceptionPercentage', 'ReceivingYards', 'ReceivingTouchdowns',
       'ReceivingYardsPerTarget', 'ReceivingYardsPerReception',
       'RushingAttempts', 'RushingYards', 'RushingYardsPerAttempt',
       'RushingTouchdowns', 'Fumbles', 'FumblesLost', 'points', 'year'],
      dtype='object')

In [14]:
indices = []
for idx,tar in enumerate(wr['ReceivingTargets']):
    if tar ==0:
        indices.append(idx)

In [16]:
wr.drop(indices, axis=0, inplace=True)

In [17]:
wr.shape

(11729, 19)

In [18]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression

In [19]:
#A function that will covert a time series database into a supervised learning database

def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = pd.DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
            cols.append(df.shift(i))
            names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

In [20]:
train  = wr
test = week1

In [21]:
wr.columns

Index(['Name', 'Team', 'Week', 'Opponent', 'ReceivingTargets', 'Receptions',
       'ReceptionPercentage', 'ReceivingYards', 'ReceivingTouchdowns',
       'ReceivingYardsPerTarget', 'ReceivingYardsPerReception',
       'RushingAttempts', 'RushingYards', 'RushingYardsPerAttempt',
       'RushingTouchdowns', 'Fumbles', 'FumblesLost', 'points', 'year'],
      dtype='object')

In [345]:
passing = ['Comp','Att','Pct','Yds','Yds/Att','TD','Int']

In [22]:
y_train = train['points']
y_test = test['points']
X_train = train.drop('points',axis=1)
X_test = test.drop('points', axis=1)

In [23]:
X_train = X_train[X_train.columns[4:12]]
X_test = X_test[X_test.columns[4:12]]

In [24]:
X_train.columns

Index(['ReceivingTargets', 'Receptions', 'ReceptionPercentage',
       'ReceivingYards', 'ReceivingTouchdowns', 'ReceivingYardsPerTarget',
       'ReceivingYardsPerReception', 'RushingAttempts'],
      dtype='object')

In [25]:
reframed = series_to_supervised(X_train.values.astype('float32'),1,1)

In [26]:
lr = LinearRegression()
lr.fit(X_train,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [27]:
coeff = pd.DataFrame(X_train.columns)
coeff['coefficients'] = lr.coef_
coeff



,0,coefficients
0,ReceivingTargets,-0.040763
1,Receptions,0.743129
2,ReceptionPercentage,-0.000388
3,ReceivingYards,0.122770
4,ReceivingTouchdowns,6.086954
5,ReceivingYardsPerTarget,0.001083
6,ReceivingYardsPerReception,-0.029576
7,RushingAttempts,0.964680


In [28]:
yhat = lr.predict(X_test)
yhat.shape

(161,)

In [29]:
def rmse(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).mean())

rmse_val = rmse(yhat, y_test)
print("RMSE error is: " + str(rmse_val))

RMSE error is: 2.81878707524


In [30]:
results = pd.DataFrame()
results['Name'] = week1['Name']
results['Predicted'] = yhat
results

,Name,Predicted
0,Antonio Brown,21.433476
1,DeAndre Hopkins,18.684725
2,Michael Thomas,18.613057
3,Julio Jones,17.476732
4,Keenan Allen,17.016170
5,AJ Green,17.083497
6,Davante Adams,15.398383
7,Stefon Diggs,15.140081
8,T.Y. Hilton,14.695691
9,Tyreek Hill,15.089147


In [89]:
results.shape

(2559, 4)

In [34]:
results.to_csv('2018_week1_wr.csv', index= False)